# Spaceship Titanic


In [ ]:
%pip install kaggle
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow[and-cuda] #  remove [and-cuda] if you don't have a GPU

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '..'
data_dir = '../data/spaceship-titanic'
!chmod 600 ../kaggle.json

In [ ]:
!kaggle competitions download -c spaceship-titanic -p {data_dir}
!unzip -o {data_dir}/spaceship-titanic.zip -d {data_dir}

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization, InputLayer, Dropout
from tensorflow.keras.regularizers import l2

/tmp/ipykernel_14440/1448122259.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-26 02:38:18.347388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 02:38:18.347456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 02:38:18.470692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable 

In [3]:
data = pd.read_csv(f'{data_dir}/train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
data = data.drop(columns=['Name', 'PassengerId'])
data = data.dropna(axis=0, how='any')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6764 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6764 non-null   object 
 1   CryoSleep     6764 non-null   object 
 2   Cabin         6764 non-null   object 
 3   Destination   6764 non-null   object 
 4   Age           6764 non-null   float64
 5   VIP           6764 non-null   object 
 6   RoomService   6764 non-null   float64
 7   FoodCourt     6764 non-null   float64
 8   ShoppingMall  6764 non-null   float64
 9   Spa           6764 non-null   float64
 10  VRDeck        6764 non-null   float64
 11  Transported   6764 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 640.7+ KB


In [5]:
def embed(data, column):
    data[column] = pd.Categorical(data[column])
    data[column] = data[column].cat.codes
    return data

def embed_all(data):
    for column in data.columns:
        if data[column].dtype == 'object' or data[column].dtype == 'bool':
            data = embed(data, column)
    return data

In [6]:
data = embed_all(data)
data.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,138,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,1854,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,1,0,1,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,1,0,1,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0,0,1856,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1


In [7]:
train, valid = train_test_split(data, test_size=0.2, random_state=42)
train_x = train.drop(columns=['Transported'], axis=1)
train_y = train['Transported']
valid_x = valid.drop(columns=['Transported'], axis=1)
valid_y = valid['Transported']

In [12]:
train_x.shape

(5411, 11)

In [ ]:
spending = pd.Series(train_x.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
age = train_x['Age']
plt.scatter(age, train_y)
plt.show(); 

In [53]:
model = Sequential(
    [
        InputLayer(input_shape=(train_x.shape[1],)),
        Normalization(),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(1)
    ]
)

model.summary()
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=train_x.shape[0] / 270 * 1000,
    decay_rate=1,
    staircase=False
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=tf.keras.metrics.BinaryAccuracy())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_10 (Normaliz  (None, 11)                23        
 ation)                                                          
                                                                 
 dense_40 (Dense)            (None, 64)                768       
                                                                 
 dropout_30 (Dropout)        (None, 64)                0         
                                                                 
 dense_41 (Dense)            (None, 32)                2080      
                                                                 
 dropout_31 (Dropout)        (None, 32)                0         
                                                                 
 dense_42 (Dense)            (None, 16)                528       
                                                     

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x.values, train_y.values)).shuffle(5411).batch(270, drop_remainder=True)
valid_ds = tf.data.Dataset.from_tensor_slices((valid_x.values, valid_y.values)).batch(270)
print(train_ds.element_spec)

In [54]:
metrics = model.fit(train_ds, validation_data=valid_ds, epochs=50, steps_per_epoch=20)

Epoch 1/50
20/20 [==============================] - 1s 12ms/step - loss: 238.9017 - binary_accuracy: 0.4880 - val_loss: 18.4144 - val_binary_accuracy: 0.5137
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 141.3708 - binary_accuracy: 0.4870 - val_loss: 13.7677 - val_binary_accuracy: 0.4745
Epoch 3/50
20/20 [==============================] - 0s 7ms/step - loss: 85.2234 - binary_accuracy: 0.5150 - val_loss: 7.7602 - val_binary_accuracy: 0.4908
Epoch 4/50
20/20 [==============================] - 0s 7ms/step - loss: 58.9673 - binary_accuracy: 0.5050 - val_loss: 3.0978 - val_binary_accuracy: 0.6201
Epoch 5/50
20/20 [==============================] - 0s 8ms/step - loss: 40.3865 - binary_accuracy: 0.5130 - val_loss: 2.6659 - val_binary_accuracy: 0.5772
Epoch 6/50
20/20 [==============================] - 0s 8ms/step - loss: 33.3187 - binary_accuracy: 0.5115 - val_loss: 1.9363 - val_binary_accuracy: 0.5188
Epoch 7/50
20/20 [==============================] - 0s 7ms/step -

In [55]:
test = pd.read_csv(f'{data_dir}/test.csv')
test.drop(columns=['Name', 'PassengerId'], inplace=True)
test = embed_all(test)

In [56]:
submission = pd.read_csv(f'{data_dir}/sample_submission.csv')
submission['Transported'] = model.predict(test)
submission.head()

134/134 [==============================] - 0s 2ms/step


,PassengerId,Transported
0,0013_01,0.141508
1,0018_01,0.085787
2,0019_01,0.190118
3,0021_01,0.037451
4,0023_01,0.158816


In [57]:
for i in range(submission.shape[0]):
    if submission.loc[i, 'Transported'] > 0.5:
        submission.loc[i, 'Transported'] = 'True'
    else:
        submission.loc[i, 'Transported'] = 'False'

submission.head()

/tmp/ipykernel_14440/2407214910.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  submission.loc[i, 'Transported'] = 'False'


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [50]:
submission.to_csv(f'{data_dir}/submission.csv', index=False)

In [51]:
!kaggle competitions submit -c spaceship-titanic -f {data_dir}/submission.csv -m "Larger nn with tf."

100%|███████████████████████████████████████| 54.9k/54.9k [00:00<00:00, 109kB/s]
Successfully submitted to Spaceship Titanic